# Exploratory analysis for federal contractors

This is just a simple exploratory analysis to get familiar with the Python language, docker containers, and Jupyter notebooks.  I've downloaded datasets from the [Federal Contractors Database](https://www.usaspending.gov/#/download_center/custom_award_data). For variable types, see the [data dictionary](http://fedspendingtransparency.github.io/dictionary-v1.1/).

[Track progress of the project on my trello board](https://trello.com/b/lZYSGp4M/federal-contractors-python)


While the goal is simply to get used to Python and some other technologies, I am interested in the specific question of ***what factors are the best predictors of minority owned companies***.  

#### Init
Read in the data and load packages

In [3]:
import pandas as pd
import numpy as np
import pandas_profiling as pp

dat = pd.read_csv('data/2017.csv', low_memory=False)

## Profiling

In [4]:
dat.shape

(72367, 225)

Since the data has 225 columns and +70k rows, I'm only going to do a profile report on a small subset of the rows.  I'm also going to save the report as an HTML file outside of this analysis.  

In [5]:
profile = pp.ProfileReport(dat.loc[0:10000])
profile.to_file(outputfile = "profiling/profile.html")

In [6]:
import matplotlib.pyplot as plt
cmt = dat.corr()

I'm mostly interested in dollars the companies recieve and the size of the company. So I'm going to make a function that checks for a certain level of correlation for the selected variable. The following cells look at:  

* Dollars Obligated
* Number of Employees
* Minority Owned Flag

In [7]:
def corMat(dd, corlv, var):
    ind = abs(dd[var]) > corlv
    return dd.loc[ind, ind]; 

In [8]:
corMat(cmt,0.05, "dollarsobligated").dollarsobligated

dollarsobligated                         1.000000
baseandexercisedoptionsvalue             0.967203
baseandalloptionsvalue                   0.861076
progsourcesubacct                       -0.076980
prime_awardee_executive1_compensation   -0.084980
prime_awardee_executive2_compensation   -0.072067
prime_awardee_executive3_compensation   -0.071098
prime_awardee_executive4_compensation   -0.071617
prime_awardee_executive5_compensation   -0.065659
Name: dollarsobligated, dtype: float64

In [9]:
corMat(cmt,0.05, "numberofemployees").numberofemployees

progsourceagency                             -0.068632
progsourcesubacct                            -0.112453
ccrexception                                 -0.357217
vendor_cd                                     0.093742
congressionaldistrict                         0.093742
placeofperformancezipcode                    -0.145658
transactionnumber                             0.100897
numberofemployees                             1.000000
veteranownedflag                             -0.077613
receivescontracts                             0.077046
issubchapterscorporation                     -0.124280
islimitedliabilitycorporation                 0.051856
ispartnershiporlimitedliabilitypartnership    0.057622
prime_awardee_executive1_compensation         0.975024
prime_awardee_executive2_compensation         0.980690
prime_awardee_executive3_compensation         0.972878
prime_awardee_executive4_compensation         0.963208
prime_awardee_executive5_compensation         0.973902
Name: numb

In [10]:
corMat(cmt,0.25, "minorityownedbusinessflag").minorityownedbusinessflag

progsourcesubacct                               -0.288274
placeofperformancezipcode                        0.255004
firm8aflag                                       0.316287
minorityownedbusinessflag                        1.000000
apaobflag                                        0.614026
baobflag                                         0.329557
naobflag                                         0.263247
haobflag                                         0.269490
isdotcertifieddisadvantagedbusinessenterprise    0.270052
prime_awardee_executive1_compensation           -0.337572
prime_awardee_executive2_compensation           -0.375064
prime_awardee_executive3_compensation           -0.398767
prime_awardee_executive4_compensation           -0.446047
prime_awardee_executive5_compensation           -0.400471
Name: minorityownedbusinessflag, dtype: float64

The minority owned business flag shows several interesting correlations:  

1. apaobflag, baobflag, naobflag, and haobflag are just subtypes of minority flags: Asian Pacific American, Black American, Native American, and Hispanic American, respectively. (Thus they're not particularly interesting. 
2. firm8aflag is for 8(a) Program Participant Organizations, which is a program for small, underpriviledged companies. 
3. DOT certified disadvantaged companies has a slightly smaller correlation. 
4. All ofthe executive compensations have a negative correlation, meaning that as executive compensation goes up, the likelihood of being minority owned is smaller.

In [11]:
# dat.prime_awardee_executive1_compensation.describe()

## Aggregation

I need to aggregate the awards based on company. Some companies have a lot of awards, so it could mess with things.  I'm also interested in adding a column that shows the count of awards for companies since that might be a good predictor of minority owned businesses. 

My hunch is that minority owned businesses are smaller than average, and have lower executive compensation/annual revenue than average. 

In [35]:
# sum of dollars obligated
grp = ['vendorname', 'minorityownedbusinessflag']
aggDollars = dat.groupby(grp)[['dollarsobligated']].agg(['sum', 'mean','count'])
aggDollars.sort_values([('vendorname')], ascending=False)[0:9]

dollarsobligated  \
                                                                      sum   
vendorname                     minorityownedbusinessflag                    
ZONES, INC.                    True                              71462.18   
ZIPPY DISPOSAL SERVICE, INC.   False                             16193.15   
ZILA WORKS LLC                 True                              99671.00   
ZIEMEK DENTAL LABORATORY, INC. False                            382110.00   
ZETEC, INC.                    False                           2045349.00   
ZEMAX, LLC                     False                            112226.00   
ZBA ARCHITECTURE PS            False                             -1074.86   
ZANOL CONSULTING INCORPORATED  False                            -38564.92   
ZAHL, ERIC                     False                             24600.00   

                                                                               
                                                                   mean count  
vendorname                     minorityownedbusinessflag                       
ZONES, INC.                    True                         6496.561818    11  
ZIPPY DISPOSAL SERVICE, INC.   False                        8096.575000     2  
ZILA WORKS LLC                 True                        99671.000000     1  
ZIEMEK DENTAL LABORATORY, INC. False                      382110.000000     1  
ZETEC, INC.                    False                       70529.275862    29  
ZEMAX, LLC                     False                        7014.125000    16  
ZBA ARCHITECTURE PS            False                       -1074.860000     1  
ZANOL CONSULTING INCORPORATED  False                      -19282.460000     2  
ZAHL, ERIC                     False                       24600.000000     1

In [36]:
# mean of prime_awardee_executive1_compensation
aggExec = dat.groupby(grp)[['prime_awardee_executive1_compensation']].agg(['mean'])
aggExec.sort_values([('vendorname')], ascending=False)[0:9]

,,prime_awardee_executive1_compensation
,,mean
vendorname,minorityownedbusinessflag,
"ZONES, INC.",True,NaN
"ZIPPY DISPOSAL SERVICE, INC.",False,NaN
ZILA WORKS LLC,True,NaN
"ZIEMEK DENTAL LABORATORY, INC.",False,NaN
"ZETEC, INC.",False,NaN
"ZEMAX, LLC",False,NaN
ZBA ARCHITECTURE PS,False,NaN
ZANOL CONSULTING INCORPORATED,False,NaN


In [117]:
# mean employeenumber
aggEmps = dat.groupby(grp)[['numberofemployees']].agg(['median'])
aggEmps.sort_values([('vendorname')], ascending=False)[0:9]

,,numberofemployees
,,median
vendorname,minorityownedbusinessflag,
"ZONES, INC.",True,1011.0
"ZIPPY DISPOSAL SERVICE, INC.",False,8.0
ZILA WORKS LLC,True,6.0
"ZIEMEK DENTAL LABORATORY, INC.",False,49.0
"ZETEC, INC.",False,10806.0
"ZEMAX, LLC",False,35.0
ZBA ARCHITECTURE PS,False,10.0
ZANOL CONSULTING INCORPORATED,False,1.0


In [118]:
# Now join the three data sets
aggs = [aggDollars, aggExec]
aggs = pd.merge(aggDollars, aggExec, on='vendorname')
aggs = pd.merge(aggs, aggEmps, on='vendorname')

aggs.sort_values([('numberofemployees', 'median')], ascending=False)[0:9]



dollarsobligated                \
                                                        sum          mean   
vendorname                                                                  
AMAZON.COM LLC                                 1.207374e+05  2.874700e+03   
NEW CINGULAR WIRELESS SERVICES, INC.           7.692881e+06  3.808357e+04   
ALSTOM GRID LLC                                2.788944e+05  9.296480e+04   
ARAMARK UNIFORM &amp; CAREER APPAREL, LLC.     3.236124e+04  3.236124e+04   
ARAMARK UNIFORM & CAREER APPAREL, LLC.         2.294863e+05  1.274924e+04   
AMAZON WEB SERVICES, INC.                      4.800000e+04  4.800000e+04   
THE BOEING COMPANY                             5.445673e+09  9.537080e+06   
INSITU, INC.                                   3.171731e+08  1.771917e+06   
BOEING COMPANY, THE                            3.668779e+06  1.048223e+05   

                                                  \
                                           count   
vendorname                                         
AMAZON.COM LLC                                42   
NEW CINGULAR WIRELESS SERVICES, INC.         202   
ALSTOM GRID LLC                                3   
ARAMARK UNIFORM &amp; CAREER APPAREL, LLC.     1   
ARAMARK UNIFORM & CAREER APPAREL, LLC.        18   
AMAZON WEB SERVICES, INC.                      1   
THE BOEING COMPANY                           571   
INSITU, INC.                                 179   
BOEING COMPANY, THE                           35   

                                           prime_awardee_executive1_compensation  \
                                                                            mean   
vendorname                                                                         
AMAZON.COM LLC                                                               NaN   
NEW CINGULAR WIRELESS SERVICES, INC.                                         NaN   
ALSTOM GRID LLC                                                              NaN   
ARAMARK UNIFORM &amp; CAREER APPAREL, LLC.                                   NaN   
ARAMARK UNIFORM & CAREER APPAREL, LLC.                                       NaN   
AMAZON WEB SERVICES, INC.                                                    NaN   
THE BOEING COMPANY                                                           NaN   
INSITU, INC.                                                                 NaN   
BOEING COMPANY, THE                                                          NaN   

                                           numberofemployees  
                                                      median  
vendorname                                                    
AMAZON.COM LLC                                     2308000.0  
NEW CINGULAR WIRELESS SERVICES, INC.                304000.0  
ALSTOM GRID LLC                                     300000.0  
ARAMARK UNIFORM &amp; CAREER APPAREL, LLC.          250000.0  
ARAMARK UNIFORM & CAREER APPAREL, LLC.              240000.0  
AMAZON WEB SERVICES, INC.                           230800.0  
THE BOEING COMPANY                                  160000.0  
INSITU, INC.                                        160000.0  
BOEING COMPANY, THE                                 159000.0

There is clearly something going on with the number of employees. I'll treat that as a caveat for now. I can't really use it, but this is all just a use case for me to get familiar with python. 

I could scrub the SEC website to find all company CIK numbers (only for publicly traded ones), then scrub through all of their 10-k forms to find their employee counts as of 12/30/2017...but that's a lot of web scraping at this stage. 

Instead, I'm going to bucket the companies by employee count size. That way I can just say that any company with more than 500,000 employees is in the largest group. 

SEC had Cingular Wireless with [65,000 employees in 2005](https://www.sec.gov/Archives/edgar/data/1130452/000095014406001479/g99532e10vk.htm). This says 304,000 (median) in 2017. [AT&T has 268,000 employees](https://www.sec.gov/Archives/edgar/data/732717/000073271717000021/ye16_10k.htm), so maybe that's what's going on.  

With that said, [Boeing is correct](https://www.sec.gov/Archives/edgar/data/12927/000001292717000006/a201612dec3110k.htm).

## Assigning Buckets to Employee Count

In [ ]:
# assign buckets to employee count

## Add Rural/Urban flag

This dataset includes zip codes and address, but that's not good as a categorical variable. I'd like to create one, but I'm going to have to figure out the best way to do it. On other projects I've had to look up population statistics from the US Census, then join those in based on FIPS codes. We'll see if that's necessary here. 

## Logistic Regression

I'd like to do a logistic regression for a few of the variables above to see if they predict minority owned business